In [1]:
import os
from urllib import parse
from urllib import request as req
import requests
import bs4
import hashlib
import selenium
import time
from PIL import Image
import io
from selenium import webdriver

In [2]:
# If you got the error "No module named 'selenium'", 
# please run the following code
#pip install selenium

In [3]:
cwd = os.getcwd()
print(cwd)

c:\Users\logan\Documents\Data Mining\Project


In [4]:
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
#DRIVER_PATH = r'C:\Users\Al\Desktop\Scraping\chromedriver2\chromedriver.exe'
#wd = webdriver.Chrome(executable_path=DRIVER_PATH)

# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH='/Users/logan/Documents/Data Mining/Project/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

C:\Users\logan\AppData\Local\Temp/ipykernel_8684/491042177.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)


In [5]:
wd.get('https://google.com')

In [6]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('Dogs')

C:\Users\logan\AppData\Local\Temp/ipykernel_8684/3184414682.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_box = wd.find_element_by_css_selector('input.gLFyf')


In [7]:
wd.quit()

In [8]:
DRIVER_PATH='/Users/logan/Documents/Data Mining/Project/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

C:\Users\logan\AppData\Local\Temp/ipykernel_8684/2548834382.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)


In [9]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            """
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")
            """
        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [10]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [11]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=1):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        if elem:
            persist_image(target_folder,elem)

In [12]:
search_term = "carpenter bee"

search_and_download(search_term=search_term,
                   driver_path = DRIVER_PATH, number_images= 100)
#for word in search_words:
#    dir_path = img_dir + word
#    if not os.path.exists(dir_path):
#        os.makedirs(dir_path)
wd.quit()

C:\Users\logan\AppData\Local\Temp/ipykernel_8684/2266906156.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
C:\Users\logan\AppData\Local\Temp/ipykernel_8684/1116780312.py:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


C:\Users\logan\AppData\Local\Temp/ipykernel_8684/1116780312.py:33: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 101 image links, done!
SUCCESS - saved https://i.pinimg.com/736x/69/d2/7b/69d27bc3519cbd46a2bcc8819ae91a6d.jpg - as ./images\carpenter_bee_sting\27fcd1fe28.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTXPYH9dnLsctdFg09qFAArs8O9W8YTyeGepg&usqp=CAU - as ./images\carpenter_bee_sting\ae5b281265.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRRktyKrOAbtiAC2owDXALBZbAdy8toUkEvRA&usqp=CAU - as ./images\carpenter_bee_sting\66884e950b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTrz-zVtWyw9Erqo3ZGWGFRq1cdEPgTcMyy6A&usqp=CAU - as ./images\carpenter_bee_sting\0a748178e7.jpg
SUCCESS - saved https://post.healthline.com/wp-content/uploads/2020/09/766x415_Home_Remedies_for_Bee_Stings_What_Works-732x415.jpg - as ./images\carpenter_bee_sting\40cb3e51a4.jpg
SUCCESS - saved https://www.thoughtco.com/thmb/xpDkwbQIKeQlG4cntWyD7dLQ2dM=/1333x1000/smart/filters:no_upscale()/150749550-58b8e2803df78c353c24a69a.jpg - as .